# 1. 自定义日志
在Django中，我们是**把有关logging的配置写到settings里面**。相应的配置及解释如下（仅供参考）。

mail_admins将信息通过邮件发送给管理员。

这里邮件的功能非常棒！并不是一个纯文本信息，而是一个html文件，和我们在浏览器看到的错误页面一模一样！要正常使用邮件功能需要像我一样配置一下上面的邮件发件人信息。我是直接去网易申请了一个邮箱。要格外注意三点：
- 1.一定要去邮件服务商开启SMTP服务；
- 2.不同的邮件服务商可能有一些特殊的设置，比如网易，会给你一个客户端授权码，这个才是密码，而不是网页的登录密码。
- 3 注意服务商有没有对发信频率的限制。

```py

# 管理员邮箱
ADMINS = (
    ('laixintao', '*******@163.com'),
)

# 非空链接，却发生404错误，发送通知MANAGERS
SEND_BROKEN_LINK_EMAILS = True
MANAGERS = ADMINS

# Email设置
EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
EMAIL_HOST = 'smtp.163.com'  # QQ邮箱SMTP服务器(邮箱需要开通SMTP服务)
EMAIL_PORT = 25  # QQ邮箱SMTP服务端口
EMAIL_HOST_USER = '**********@163.com'  # 我的邮箱帐号
EMAIL_HOST_PASSWORD = '**************'  # 授权码
EMAIL_SUBJECT_PREFIX = 'website'  # 为邮件标题的前缀,默认是'[django]'
EMAIL_USE_TLS = True  # 开启安全链接
DEFAULT_FROM_EMAIL = SERVER_EMAIL = EMAIL_HOST_USER  # 设置发件人

# logging日志配置
LOGGING = {
    'version': 1,
    'disable_existing_loggers': True,
    'formatters': {  # 日志格式
        'standard': {
            'format': '%(asctime)s [%(threadName)s:%(thread)d] [%(name)s:%(lineno)d] [%(module)s:%(funcName)s] [%(levelname)s]- %(message)s'}
    },
    'filters': {  # 过滤器
        'require_debug_false': {
            '()': 'django.utils.log.RequireDebugFalse',
        }
    },
    'handlers': {  # 处理器
        'null': {
            'level': 'DEBUG',
            'class': 'logging.NullHandler',
        },
        'mail_admins': {  # 发送邮件通知管理员
            'level': 'ERROR',
            'class': 'django.utils.log.AdminEmailHandler',
            'filters': ['require_debug_false'],  # 仅当 DEBUG = False 时才发送邮件
            'include_html': True,
        },
        'debug': {  # 记录到日志文件(需要创建对应的目录，否则会出错)
            'level': 'DEBUG',
            'class': 'logging.handlers.RotatingFileHandler',
            'filename': os.path.join(BASE_DIR, "log", 'debug.log'),  # 日志输出文件
            'maxBytes': 1024 * 1024 * 5,  # 文件大小
            'backupCount': 5,  # 备份份数
            'formatter': 'standard',  # 使用哪种formatters日志格式
        },
        'console': {  # 输出到控制台
            'level': 'DEBUG',
            'class': 'logging.StreamHandler',
            'formatter': 'standard',
        },
    },
    'loggers': {  # logging管理器
        'django': {
            'handlers': ['console'],
            'level': 'DEBUG',
            'propagate': False
        },
        'django.request': {
            'handlers': ['debug', 'mail_admins'],
            'level': 'ERROR',
            'propagate': True,
        },
        # 对于不在 ALLOWED_HOSTS 中的请求不发送报错邮件
        'django.security.DisallowedHost': {
            'handlers': ['null'],
            'propagate': False,
        },
    }
}

```

# 2. Django 内置日志

当服务器500错误的时候，普通日志只会记录一行500的request信息，并不会记录详细的报错定位

>[ERROR] 2019-06-12 15:07:03,597 "GET /api/v1/test/ HTTP/1.1" 500 74196

1. 需要添加一个在日志中记录详细错误信息的middleware

```py
# -*- coding: UTF-8 -*-
import logging
 
logger = logging.getLogger('default')
 
 
class ExceptionLoggingMiddleware(object):
  def process_exception(self, request, exception):
    import traceback
    logger.error(traceback.format_exc())
```

2. 将此中间件添加到settings的MIDDLEWARE_CLASSES中

```py
MIDDLEWARE_CLASSES = (
    'utils.my_middleware.ExceptionLoggingMiddleware',
)
```

3. 使用Django logger

```py
'loggers': {
  'django': {
    'handlers': ['file', 'console'],
    'level': 'INFO'
  },
```

4. 最后，不要忘了给日志的路径响应的权限。比如Apache2服务器，就需要给www-data写权限：

```sh
sudo chown -R [yourname]:www-data [log]
sudo chmod -R g+s [log]
```